In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from config import g_key

#configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# Loan CSV file generated from WeatherPy Folder
vacation_df = "../WeatherPy/output_data/cities.csv"
vacation_df = pd.read_csv(vacation_df)
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.65,115.33,54.00,54,0,10.56,AU,1603916711
1,1,Port Alfred,-33.59,26.89,66.22,86,86,2.71,ZA,1603916711
2,2,Mamit,23.93,92.48,68.05,99,21,1.30,IN,1603916711
3,3,Carnarvon,-24.87,113.63,68.00,94,0,13.87,AU,1603916711
4,4,Tiksi,71.69,128.87,0.70,91,99,18.32,RU,1603916711
...,...,...,...,...,...,...,...,...,...,...
536,536,Davenda,53.56,119.32,19.99,85,6,0.51,RU,1603916747
537,537,Salalah,17.02,54.09,82.40,74,40,2.24,OM,1603916747
538,538,Gijang,35.24,129.21,46.40,65,0,2.24,KR,1603916747
539,539,North Bend,43.41,-124.22,73.00,63,1,12.75,US,1603916747


In [3]:
# Heatmap displaying Humidity for every city in vacation_df

locations = vacation_df[["Lat", "Lng"]].astype(float)
humidity = vacation_df["Humidity"].astype(float)

fig = gmaps.figure(center=(0.000, 0.000), zoom_level=1.9)

#heatLayer_humidity = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius = 3)

fig.add_layer(gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius = 3))

fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrowing down the DataFrame to find ideal weather condition. 
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Dropping any rows that don't contain all three conditions. 

new_vacation_df = pd.DataFrame(vacation_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_vacation_df["Max Temp"] <= 80) & (new_vacation_df["Max Temp"] > 70)
wind_speed = new_vacation_df["Wind Speed"] < 10
cloudiness = new_vacation_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_vacation_df[max_temp & wind_speed & cloudiness]
new_vacation_df

,City,Max Temp,Wind Speed,Cloudiness
0,Busselton,54.00,10.56,0
1,Port Alfred,66.22,2.71,86
2,Mamit,68.05,1.30,21
3,Carnarvon,68.00,13.87,0
4,Tiksi,0.70,18.32,99
...,...,...,...,...
536,Davenda,19.99,0.51,6
537,Salalah,82.40,2.24,40
538,Gijang,46.40,2.24,0
539,North Bend,73.00,12.75,1


In [5]:
hotel_df = pd.DataFrame(vacation_df, columns=["City", "Country", "Lat", "Lng"])
hotel_df

,City,Country,Lat,Lng
0,Busselton,AU,-33.65,115.33
1,Port Alfred,ZA,-33.59,26.89
2,Mamit,IN,23.93,92.48
3,Carnarvon,AU,-24.87,113.63
4,Tiksi,RU,71.69,128.87
...,...,...,...,...
536,Davenda,RU,53.56,119.32
537,Salalah,OM,17.02,54.09
538,Gijang,KR,35.24,129.21
539,North Bend,US,43.41,-124.22


In [6]:
#set up lists for hotel information
hotelName = []
hotelAddress = []

for index, row in hotel_df.iterrows():
    #current info
    currentCity = row["City"]
    currentCountry = row["Country"]
    
    #geocoordinates
    target_coordinates = f"{row['Lat']},{row['Lng']}"
    target_radius = 5000 
    target_type = "hotel"
    target_keyword = "hotel"

    #set parameters
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "keyword": target_keyword,
        "key": g_key
    }
    
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #run request using params dictionary
    response = requests.get(base_url,params=params).json()
    #simplify finding results
    results = response['results']
     
    try:
        hotelName.append(results[0]['name'])
        hotelAddress.append(results[0]['vicinity'])
        print(f"Processeing Record for a {target_keyword} in {currentCity}, {currentCountry}")
    except:
        hotelName.append(np.nan)
        hotelAddress.append(np.nan)
        print(f"No results/{target_keyword} found in {currentCity}, {currentCountry}. Skipping...")


Processeing Record for a hotel in Busselton, AU
Processeing Record for a hotel in Port Alfred, ZA
Processeing Record for a hotel in Mamit, IN
Processeing Record for a hotel in Carnarvon, AU
Processeing Record for a hotel in Tiksi, RU
No results/hotel found in Labuhan, ID. Skipping...
Processeing Record for a hotel in Mataura, NZ
Processeing Record for a hotel in Kahului, US
Processeing Record for a hotel in Atuona, PF
Processeing Record for a hotel in Saint-Philippe, RE
Processeing Record for a hotel in Havre-St-Pierre, CA
Processeing Record for a hotel in Slavyanka, RU
Processeing Record for a hotel in Rikitea, PF
Processeing Record for a hotel in Ushuaia, AR
Processeing Record for a hotel in Dubovka, RU
Processeing Record for a hotel in Andenes, NO
Processeing Record for a hotel in Narsaq, GL
Processeing Record for a hotel in Pozo Colorado, PY
Processeing Record for a hotel in Bathsheba, BB
Processeing Record for a hotel in Hermanus, ZA
Processeing Record for a hotel in Iquitos, PE
P

Processeing Record for a hotel in Los Baños, PH
Processeing Record for a hotel in Bouna, CI
Processeing Record for a hotel in Banda Aceh, ID
No results/hotel found in Kruisfontein, ZA. Skipping...
Processeing Record for a hotel in Alofi, NU
Processeing Record for a hotel in Lipin Bor, RU
Processeing Record for a hotel in De-Kastri, RU
Processeing Record for a hotel in Spokane, US
Processeing Record for a hotel in Lebu, CL
Processeing Record for a hotel in Clyde River, CA
Processeing Record for a hotel in Te Anau, NZ
Processeing Record for a hotel in Giohar, SO
Processeing Record for a hotel in Ţahţā, EG
Processeing Record for a hotel in Yangi Marg`ilon, UZ
No results/hotel found in Kamenka, RU. Skipping...
Processeing Record for a hotel in Hammerfest, NO
Processeing Record for a hotel in Mountain Home, US
Processeing Record for a hotel in Prince George, CA
Processeing Record for a hotel in Vao, NC
No results/hotel found in Lavrentiya, RU. Skipping...
Processeing Record for a hotel in C

No results/hotel found in Birao, CF. Skipping...
Processeing Record for a hotel in Boa Vista, BR
Processeing Record for a hotel in Fayaoué, NC
Processeing Record for a hotel in Marawi, PH
Processeing Record for a hotel in Tupik, RU
No results/hotel found in Toora-Khem, RU. Skipping...
Processeing Record for a hotel in Amilly, FR
Processeing Record for a hotel in Caucayá, CO
Processeing Record for a hotel in Rāvar, IR
Processeing Record for a hotel in Gempol, ID
Processeing Record for a hotel in Swellendam, ZA
Processeing Record for a hotel in Kununurra, AU
No results/hotel found in Aketi, CD. Skipping...
No results/hotel found in Panabá, MX. Skipping...
Processeing Record for a hotel in Kloulklubed, PW
Processeing Record for a hotel in Moundou, TD
Processeing Record for a hotel in São João da Barra, BR
Processeing Record for a hotel in Ailigandí, PA
No results/hotel found in Etchojoa, MX. Skipping...
Processeing Record for a hotel in Ouésso, CG
Processeing Record for a hotel in Taksimo

Processeing Record for a hotel in Broome, US
Processeing Record for a hotel in Vanimo, PG
Processeing Record for a hotel in Sri Aman, MY
No results/hotel found in Rodolfo Sánchez Taboada, MX. Skipping...
No results/hotel found in Poum, NC. Skipping...
Processeing Record for a hotel in Saint-Pierre, RE
Processeing Record for a hotel in Uturoa, PF
Processeing Record for a hotel in Inta, RU
Processeing Record for a hotel in Acapulco de Juárez, MX
No results/hotel found in Yerbogachën, RU. Skipping...
Processeing Record for a hotel in Kribi, CM
Processeing Record for a hotel in Presidencia Roque Sáenz Peña, AR
Processeing Record for a hotel in Sanandij, IR
Processeing Record for a hotel in Santa Cruz de la Sierra, BO
Processeing Record for a hotel in Mossamedes, AO
No results/hotel found in Marzuq, YE. Skipping...
Processeing Record for a hotel in Verkhnyaya Maksakovka, RU
No results/hotel found in Phayao, TH. Skipping...
Processeing Record for a hotel in Juneau, US
Processeing Record for 

In [7]:
#add new information into hotel_df
hotel_df["Hotel Name"] = hotelName
hotel_df["Hotel Address"] = hotelAddress
#hotel_df = hotel_df.dropna()

hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel Address
0,Busselton,AU,-33.65,115.33,Broadwater Resort,"11 Holgate Rd, Broadwater"
1,Port Alfred,ZA,-33.59,26.89,Royal St Andrews Hotel,"19 St Andrews Rd, Port Alfred"
2,Mamit,IN,23.93,92.48,Boo Boo Guest House,"New Mamit, Mamit"
3,Carnarvon,AU,-24.87,113.63,Hospitality Carnarvon,"6 West St, South Carnarvon"
4,Tiksi,RU,71.69,128.87,Arktika,"Ulitsa Polyarnoy Aviatsii, д. 4, Tiksi"
...,...,...,...,...,...,...
536,Davenda,RU,53.56,119.32,NaN,NaN
537,Salalah,OM,17.02,54.09,BAITI Hotel Apartments,"Unnamed Road, Salalah"
538,Gijang,KR,35.24,129.21,Hilton Busan,"268-32 Gijanghaean-ro, Gijang-eup, Gijang-gun"
539,North Bend,US,43.41,-124.22,The Mill Casino • Hotel & RV Park,"3201 Tremont Ave, North Bend"


In [8]:
#remove NaN rows
#did not drop indexes for locations without hotel found - can see how many hotels are found for top locations
hotelFound_df = hotel_df.dropna()
hotelFound_df

,City,Country,Lat,Lng,Hotel Name,Hotel Address
0,Busselton,AU,-33.65,115.33,Broadwater Resort,"11 Holgate Rd, Broadwater"
1,Port Alfred,ZA,-33.59,26.89,Royal St Andrews Hotel,"19 St Andrews Rd, Port Alfred"
2,Mamit,IN,23.93,92.48,Boo Boo Guest House,"New Mamit, Mamit"
3,Carnarvon,AU,-24.87,113.63,Hospitality Carnarvon,"6 West St, South Carnarvon"
4,Tiksi,RU,71.69,128.87,Arktika,"Ulitsa Polyarnoy Aviatsii, д. 4, Tiksi"
...,...,...,...,...,...,...
532,Whitehorse,CA,60.72,-135.05,Best Western Gold Rush Inn,"411 Main St, Whitehorse"
535,Quesnel,CA,53.00,-122.50,Sandman Hotel Quesnel,"940 Chew Rd, Quesnel"
537,Salalah,OM,17.02,54.09,BAITI Hotel Apartments,"Unnamed Road, Salalah"
538,Gijang,KR,35.24,129.21,Hilton Busan,"268-32 Gijanghaean-ro, Gijang-eup, Gijang-gun"


In [13]:

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotelFound_info = [info_box_template.format(**row) for index, row in hotelFound_df.iterrows()]
# locations = hotelFound_df[["Lat", "Lng"]]

In [ ]:
# # Add marker layer ontop of heat map
# markers = gmaps.marker_layer(locations, info_box_content=hotelFound_info)
# fig.add_layer(markers)

# # Display figure
# fig